# 🥇 Gold Layer – Business-Level, Curated Data

The Gold layer contains fully refined, business-ready datasets designed for analytics, reporting, and machine learning.  
It represents the highest-quality, most trusted version of the data in the Lakehouse.

### Key characteristics
- Data is **aggregated**, **modeled**, and aligned to business entities
- Tables are optimized for BI tools, dashboards, and ML features
- Business logic and KPIs are applied consistently
- Surfaces only the fields required for consumption
- High performance and low-latency access

### Typical tasks in this layer
- Build star schemas, fact tables, and dimension tables
- Apply business metrics (e.g., revenue, churn, retention, conversion)
- Create feature tables for ML models
- Produce curated Delta tables for dashboards and downstream applications
- Join multiple Silver tables into unified business views


Create gold.fact_orders table

In [0]:
from pyspark.sql.functions import col

silver_df = spark.table("silver.sales_orders")

fact_orders_df = silver_df.select(
    col("order_number"),
    col("customer_id"),
    col("customer_name"),
    col("order_date"),
    col("order_year"),
    col("order_month"),
    col("number_of_line_items")
)

fact_orders_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold.fact_orders")


In [0]:
%sql
SELECT COUNT(*) FROM gold.fact_orders;


COUNT(*)
4000


In [0]:
%sql
Select * from gold.fact_orders Limit 10;

order_number,customer_id,customer_name,order_date,order_year,order_month,number_of_line_items
317568493,7442821,"FITZMAURICE, PATRICK",2019-08-13,2019,8,3
317568652,17412651,"DUNLEAVY-GERSTER, MONICA T",2019-08-17,2019,8,2
317570136,4952777,flying point digital,2019-09-26,2019,9,1
317570871,19580290,genesis electronics recycling,2019-10-17,2019,10,2
317571868,30544967,"GARRETT, BRADLEY N",2019-11-11,2019,11,2
317569182,417833,"VILARETE, MARIA A",2019-09-01,2019,9,1
317569266,17406352,sports pro physical therapy,2019-09-03,2019,9,2
317569585,17728997,helios electronics limited,2019-09-12,2019,9,1
317571876,10393805,"atlanta digital studio, llc",2019-11-11,2019,11,1
317569317,10393805,"atlanta digital studio, llc",2019-09-04,2019,9,1


In [0]:
from pyspark.sql.functions import explode, from_json
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

product_schema = ArrayType(
    StructType([
        StructField("id", StringType()),
        StructField("name", StringType()),
        StructField("price", StringType()),
        StructField("qty", StringType()),
        StructField("curr", StringType())
    ])
)

products_df = silver_df.withColumn(
    "product",
    explode(from_json(col("ordered_products"), product_schema))
)


Flatten

In [0]:
product_sales_df = products_df.select(
    col("order_number"),
    col("order_date"),
    col("product.id").alias("product_id"),
    col("product.name").alias("product_name"),
    col("product.qty").cast("int").alias("quantity"),
    col("product.price").cast("double").alias("unit_price")
)


Save

In [0]:
product_sales_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold.product_sales")


Validation

In [0]:
%sql
SELECT product_name, SUM(quantity) AS units_sold
FROM gold.product_sales
GROUP BY product_name
ORDER BY units_sold DESC;


product_name,units_sold
Cyber-shot DSC-WX220 Digital Camera (Black),1024
Zamaha - MusicCast 5.1-Ch. 4K Ultra HD A/V Home Theater Receiver - Black,1021
Zamaha - AVENTAGE 7.2-Ch. 4K Ultra HD A/V Home Theater Receiver - Black,996
"15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)",984
CD-C600 5-Disc CD Changer,977
Adventura SH 140 II Shoulder Bag (Black),973
Details About Mogitech G920 Xbox Driving Force Racing Wheel For Xbox One And Pc (941000121),971
"Elpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black""",952
Mogitech Ultrathin Touch Mouse T630 for Windows 8 Touch Gestures,941
Elpine - Rear View Camera - Black,924


Create gold.daily_kpis table

In [0]:
%sql
CREATE OR REPLACE TABLE gold.daily_kpis AS
SELECT
    order_date,
    COUNT(DISTINCT order_number) AS total_orders,
    COUNT(DISTINCT customer_id) AS total_customers,
    SUM(number_of_line_items) AS total_line_items
FROM gold.fact_orders
GROUP BY order_date
ORDER BY order_date;


num_affected_rows,num_inserted_rows


In [0]:
%sql
Select * from gold.daily_kpis Limit 10;

order_date,total_orders,total_customers,total_line_items
null,45,32,90
2019-08-01,38,35,78
2019-08-02,48,44,102
2019-08-03,20,20,48
2019-08-04,24,22,55
2019-08-05,45,39,93
2019-08-06,71,59,162
2019-08-07,40,38,71
2019-08-08,35,32,81
2019-08-09,34,31,65


Create gold.promo_performance table

In [0]:
promo_schema = ArrayType(
    StructType([
        StructField("promo_item", StringType()),
        StructField("promo_disc", StringType()),
        StructField("promo_qty", StringType())
    ])
)

promo_df = silver_df.withColumn(
    "promo",
    explode(from_json(col("promo_info"), promo_schema))
).filter(col("promo").isNotNull())

promo_gold_df = promo_df.select(
    col("order_number"),
    col("promo.promo_item").alias("product_id"),
    col("promo.promo_disc").cast("double").alias("discount"),
    col("promo.promo_qty").cast("int").alias("promo_qty")
)

promo_gold_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold.promo_performance")


In [0]:
%sql
Select * from gold.promo_performance limit 10;

order_number,product_id,discount,promo_qty
317568652,AVpfMVD-ilAPnD_xW6bu,0.03,1
317570136,AVpfMVD-ilAPnD_xW6bu,0.03,10
317570871,AVpfMVD-ilAPnD_xW6bu,0.03,3
317569317,AVqVGUFCv8e3D1O-ldFF,0.05,1
317569030,AVpfMVD-ilAPnD_xW6bu,0.03,3
317571749,AVphq88q1cnluZ0-FFwx,0.05,3
317569183,AVpfMVD-ilAPnD_xW6bu,0.03,3
317569485,AVpiHEE31cnluZ0-J8jJ,0.07,1
317569515,AVphq88q1cnluZ0-FFwx,0.05,3
317569706,AVpfZaCp1cnluZ0-kDV9,0.05,2


Answering the business question

In [0]:
from pyspark.sql.functions import explode, col, from_json, sum as _sum, when
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

product_schema = ArrayType(
    StructType([
        StructField("id", StringType()),
        StructField("name", StringType()),
        StructField("price", StringType()),
        StructField("qty", StringType()),
        StructField("promotion_info", StringType())
    ])
)

products_df = (
    spark.table("silver.sales_orders")
    .withColumn("product", explode(from_json(col("ordered_products"), product_schema)))
)


Calculate revenue per order

In [0]:
revenue_df = (
    products_df
    .withColumn("price", col("product.price").cast("double"))
    .withColumn("qty", col("product.qty").cast("int"))
    .withColumn("line_revenue", col("price") * col("qty"))
)


In [0]:
gold_revenue_df = (
    revenue_df
    .groupBy("order_number", "order_date", "customer_id")
    .agg(
        _sum("qty").alias("total_items"),
        _sum("line_revenue").alias("total_revenue")
    )
)


Join tables

In [0]:
promo_flag_df = (
    spark.table("silver.sales_orders")
    .select("order_number", "promo_info")
    .withColumn("promo_applied", col("promo_info") != "[]")
)

gold_revenue_df = (
    gold_revenue_df
    .join(promo_flag_df, "order_number", "left")
)


Write to able

In [0]:
spark.sql("DROP TABLE IF EXISTS gold.order_revenue")

gold_revenue_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold.order_revenue")


Validation Checks

In [0]:
%sql
SELECT *
FROM gold.order_revenue
ORDER BY total_revenue DESC
LIMIT 10;


order_number,order_date,customer_id,total_items,total_revenue,promo_info,promo_applied
317569798,2019-09-17,782583,13,32851.0,"[{""promo_disc"":0.05,""promo_id"":""1"",""promo_item"":""AVqVGUFCv8e3D1O-ldFF"",""promo_qty"":""8""}]",true
317570373,2019-10-03,11460859,20,29536.0,[],false
317569397,2019-09-06,22100013,15,26223.0,"[{""promo_disc"":0.05,""promo_id"":""1"",""promo_item"":""AVphq88q1cnluZ0-FFwx"",""promo_qty"":""2""}]",true
317570744,2019-10-14,14739141,16,25175.0,[],false
317568125,2019-08-04,21823196,19,24891.0,[],false
317568805,2019-08-21,868848,20,24609.0,"[{""promo_disc"":0.05,""promo_id"":""1"",""promo_item"":""AVpfZaCp1cnluZ0-kDV9"",""promo_qty"":""1""}]",true
317570528,2019-10-08,28447330,13,24496.0,"[{""promo_disc"":0.05,""promo_id"":""1"",""promo_item"":""AVqVGUFCv8e3D1O-ldFF"",""promo_qty"":""6""}]",true
317570826,2019-10-15,3037810,17,23945.0,[],false
317571923,2019-11-12,1542007,9,23715.0,"[{""promo_disc"":0.05,""promo_id"":""1"",""promo_item"":""AVqVGUFCv8e3D1O-ldFF"",""promo_qty"":""3""}]",true
317571616,2019-11-05,17796669,14,22152.0,[],false


In [0]:
%sql
SELECT COUNT(*) AS bad_orders
FROM silver.sales_orders
WHERE number_of_line_items <= 0;


bad_orders
0


In [0]:
%sql
SELECT COUNT(*) AS empty_orders
FROM silver.sales_orders
WHERE ordered_products = '[]' OR ordered_products IS NULL;


empty_orders
0


In [0]:
%sql
SELECT COUNT(*) AS invalid_revenue_orders
FROM gold.order_revenue
WHERE total_revenue <= 0;


invalid_revenue_orders
0
